ValueError: ignored

In [ ]:
datasetPath = "./drive/MyDrive/dataset/"
parameterPath = "./drive/MyDrive/parameters"

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import numpy as np

In [4]:
cifar10_train = dataset.CIFAR10(root = datasetPath,
                            train = True,
                            transform = transform.ToTensor(),
                            download = True)

cifar10_test = dataset.CIFAR10(root = datasetPath,
                           train = False,
                           transform = transform.ToTensor(),
                           download = True)

HTTPError: ignored

In [ ]:
class VGG (nn.Module):
    def __init__(self): # 신경망 구성요소 정의
        super(VGG, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3,  padding=1)
        self.conv5 = nn.Conv2d(in_channels=128,out_channels=256, kernel_size=3,  padding=1)
        self.conv6 = nn.Conv2d(in_channels=256,out_channels=256, kernel_size=3,  padding=1)

        
        self.fc1 = nn.Linear(in_features=4096, out_features=2000)
        self.fc2 = nn.Linear(in_features=2000, out_features=1000)
        self.fc3 = nn.Linear(in_features=1000, out_features=10)
        self.relu = nn.ReLU()
        self.maxPool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm1d(2000)
        self.bn2 = nn.BatchNorm1d(1000)

    def forward(self, x): # 신경망 동작 순서 (입출력 구조) 정의
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.maxPool2d(out)

        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        out = self.maxPool2d(out)

        out = self.relu(self.conv5(out))
        out = self.relu(self.conv6(out))
        out = self.maxPool2d(out)

        out = out.view(-1, 4096) #feature map 평탄화
        out = self.relu(self.bn1(self.fc1(out)))
        out = self.relu(self.bn2(self.fc2(out)))
        out = self.fc3(out)
        return out


In [ ]:
batch_size = 100
learning_rate = 0.1
training_epoch = 20
loss_function = nn.CrossEntropyLoss()
network = VGG()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
!nvidia-smi

In [5]:
network.train()
network = network.to('cuda:0')
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = len(data_loader)

    for img, label in data_loader:
        
        img = img.to('cuda:0')
        label = label.to('cuda:0')
        pred = network(img)
        loss = loss_function(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')      

NameError: ignored

In [ ]:
torch.save(network.state_dict(), parameterPath+"vgg_cifar10.pth")

In [6]:
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data , (0, 3, 1, 2))) /255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
    prediction = network(img_test) # 전체 test data를 한번에 계산

correct_prediction = torch.argmax(prediction, 1) == label_test
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy.item())

NameError: ignored